In [266]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing import image

In [267]:
data_dir = 'DataSets'
batch_size = 32

train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    #width_shift_range=0.1,      # Small horizontal shifts are okay
    #height_shift_range=0.05,    # Small vertical shifts are okay
    # No rotation, zoom, or flip since it’s a spectrogram
    validation_split=0.1       # Split for validation
)

In [268]:
#training data generator from the given directory followed by data augmentation
train_generator = train_datagen.flow_from_directory(
    directory=data_dir,
    target_size=(128, 128),     # Resize to model input size
    batch_size=32,
    class_mode='categorical',
    subset='training'
)



validation_generator = train_datagen.flow_from_directory(
    directory=data_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

Found 542 images belonging to 6 classes.
Found 60 images belonging to 6 classes.


In [269]:
#CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128,128, 3)),
    MaxPooling2D((2, 2)),
    
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),  
        Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),  
        


    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(units = 6  , activation='softmax')  # Softmax for multi-class classification
])

# Step 3: Compile the Model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])


/Users/tejasbaid/Library/Python/3.9/lib/python/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [270]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=validation_generator,
    validation_batch_size=validation_generator.samples // batch_size,
    epochs=50,
    callbacks=[]
)


Epoch 1/50


/Users/tejasbaid/Library/Python/3.9/lib/python/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


16/16 ━━━━━━━━━━━━━━━━━━━━ 9s 400ms/step - accuracy: 0.1633 - loss: 1.8149 - val_accuracy: 0.1833 - val_loss: 1.7476
Epoch 2/50
 1/16 ━━━━━━━━━━━━━━━━━━━━ 2s 155ms/step - accuracy: 0.1250 - loss: 1.7525

/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/contextlib.py:135: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(type, value, traceback)


16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 69ms/step - accuracy: 0.1250 - loss: 1.7525 - val_accuracy: 0.3667 - val_loss: 1.7427
Epoch 3/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 8s 326ms/step - accuracy: 0.3192 - loss: 1.6887 - val_accuracy: 0.6667 - val_loss: 1.4449
Epoch 4/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 42ms/step - accuracy: 0.3125 - loss: 1.6142 - val_accuracy: 0.5000 - val_loss: 1.4297
Epoch 5/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 8s 317ms/step - accuracy: 0.4564 - loss: 1.3112 - val_accuracy: 0.7167 - val_loss: 0.9703
Epoch 6/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 42ms/step - accuracy: 0.5625 - loss: 1.2091 - val_accuracy: 0.3833 - val_loss: 1.1509
Epoch 7/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 8s 330ms/step - accuracy: 0.5892 - loss: 1.0398 - val_accuracy: 0.9167 - val_loss: 0.6239
Epoch 8/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 46ms/step - accuracy: 0.6875 - loss: 0.7681 - val_accuracy: 0.7500 - val_loss: 0.6230
Epoch 9/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 9s 408ms/step - accuracy: 0.5983 - loss: 0.8530 - val_accuracy: 0.9000 - val_loss

KeyboardInterrupt: 

In [271]:
def preprocess_final_image(img_path, target_size=(128, 128)):
    img = image.load_img(img_path, target_size=target_size)  # Load and resize image
    img_array = image.img_to_array(img)  # Convert to array
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array /= 255.0  # Normalize to [0, 1] range if needed
    return img_array

In [272]:
img_path = '02_national_anthem.png'
img = preprocess_final_image(img_path)
predictions = model.predict(img)
predicted_class = np.argmax(predictions, axis=1)
print(f"Predicted class: {predicted_class}")

print("Probabilities for each class:", predictions)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
Predicted class: [2]
Probabilities for each class: [[8.6849101e-04 1.4310517e-07 9.1546756e-01 2.2958050e-06 8.3661519e-02
  1.1712210e-10]]


In [274]:
folder_path = 'test-2'
max_probabilities = []
output_file = 'model_predictions_output.txt'
class_indices = train_generator.class_indices

# Reverse the dictionary to map indices back to class labels
class_labels = {v: k for k, v in class_indices.items()}
'''
for img_filename in os.listdir(folder_path):
    # Check if the file is an image (optional)
    if img_filename.endswith(('.png', '.jpg', '.jpeg')):
        img_path = os.path.join(folder_path, img_filename)
        
        # Preprocess the image and make predictions
        img = preprocess_final_image(img_path)
        predictions = model.predict(img)
        max_probability = np.max(predictions, axis=1)[0]  
        # Get the predicted class
        predicted_class = np.argmax(predictions, axis=1)
        max_probabilities.append(max_probability)
        # Print or store the results
        print(f"File: {img_filename}")
        print(f"Predicted class: {predicted_class[0]}")
        print("Probabilities for each class:", predictions[0])
        print(f"Confidence (Max Probability): {max_probability}")

average_confidence = np.mean(max_probabilities)
print(f"Average model confidence {average_confidence}")
'''
# Open the output file for writing
national_anthem = 0
number_of_predictions = 0
with open(output_file, 'w') as file:
    # Loop through each image file in the folder
    for img_filename in os.listdir(folder_path):
        # Check if the file is an image (optional)
        if img_filename.endswith(('.png', '.jpg', '.jpeg')):
            img_path = os.path.join(folder_path, img_filename)
            
            # Preprocess the image and make predictions
            img = preprocess_final_image(img_path)
            predictions = model.predict(img)
            max_probability = np.max(predictions, axis=1)[0]  
            predicted_class = np.argmax(predictions, axis=1)
            
            # Store the max probability
            max_probabilities.append(max_probability)
            predicted_label = class_labels[predicted_class[0]]
            if predicted_class[0]==2:
                national_anthem = national_anthem+1
            # Write results to file
            if max_probability>0.8:
                file.write(f"File: {img_filename}\n")
                file.write(f"predicted class label :{predicted_label}\n")
                file.write(f"Probabilities for each class: {predictions[0]}\n")
                file.write(f"Confidence (Max Probability): {max_probability}\n\n")
                number_of_predictions = number_of_predictions+1
            else :
                file.write(f"File: {img_filename}\n")
                file.write(f"predicted class label :inconclusive prediction")
                file.write(f"Probabilities for each class: {predictions[0]}\n")
                file.write(f"Confidence (Max Probability): {max_probability}\n\n")
            

    # Calculate and write the average confidence to the file
    average_confidence = np.mean(max_probabilities)
    file.write(f"\nAverage model confidence across all images: {average_confidence}\n")
print(national_anthem)
print(f"number_of_predictions : {number_of_predictions}")
print(f"Results saved to {output_file}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━

In [240]:
class_indices = train_generator.class_indices

# Reverse the dictionary to map indices back to class labels
class_labels = {v: k for k, v in class_indices.items()}

# Now find the predicted class label

predicted_label = class_labels[predicted_class[0]]
print(f"Predicted class label: {predicted_label}")

Predicted class label: National Anthem_masked
